In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy import stats,sparse
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma
import pickle as pickle
import xgboost as xgb
import time
from pandas import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 

import json
from sklearn.metrics import roc_curve, auc
from re import sub
from collections import defaultdict
import scipy as sp

from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn import metrics

#from ModelClassifier import PAClassifier,SGDSVMClassifier,XGBoostClassifier

In [2]:
path_to_results = 'FinalPrediction/'

test_ID = pickle.load(open(path_to_results+'xtest_ID.pkl','rb'))    
y = pickle.load(open(path_to_results+'ytrain2.dat','rb'))  # ground-truth label of training data
print(y.shape)

(145231L,)


In [3]:
#unused: 'svm-0p752773_wl_', "pa-283_","pa-335_","pa-0p75277_wl_",
#"sgd-283_","sgd-335_",'sgd-0p768996_wl_', 'sgd_0p769531_wl_',
#          "rf-335_","rf-all_",
#          "xgb-665_","xgb-933_","xgb-all_","xgb_0p770751_wl_",

results=["sgd_0p771702_wl_","sgd-0p770277_wl_","sgd_0p770356_wl_",
         "xgb_0p784886_wl_","xgb_0p785604_wl_", "xgb_0p785155_wl_","xgb_0p785468_wl_",
         "xgb_0p785787_wl_","xgb_0p785890_wl_","xgb_0p786040_wl_","xgb_0p786290_wl_",
         "xgb_lr_0p777824_wl_","xgb_lr_0p777125_50Segments_wl_","xgb_lr_0p776415_wl_",
         "xgb_0p780004_sl_", "xgb_0p781263_sl_", "xgb_0p785606_sl_", "xgb_0p786338_sl_", 
         "xgb_0p786733_sl_", "xgb_0p787197_sl_", "xgb_0p787252_sl_", "xgb_0p787437_sl_", 
         "xgb_0p787608_sl_", "xgb_0p787632_sl_", "xgb_0p787695_sl_", "xgb_0p787790_sl_", 
         "xgb_0p779609_sl_", "xgb_0p779270_sl_", "xgb_0p778827_sl_", "xgb_0p778479_sl_", 
         "xgb_0p778008_sl_", "xgb_0p777976_sl_", "xgb_0p777858_sl_", "xgb_0p777086_sl_",
         "xgb_0p777080_sl_"
        ]

In [4]:
len(results)

35

In [5]:
nameypreds_train = 'ypredtrain.pkl'
nameypreds_test = 'ypredtest.pkl'

preds_train=[]
preds_test =[]

for i, model in enumerate(results):
    ypred_train = list(pickle.load(open(path_to_results + model + nameypreds_train, 'rb')))
#     print('Model-%d: The min and max of model %s ypred_train is: %f, %f'%(i+1, model,np.array(ypred_train).min(),np.array(ypred_train).max()))
#     print('Model-%d: The AUC score for model %s is: %f\n'%(i+1, model,metrics.roc_auc_score(y, ypred_train)))
    preds_train.append(ypred_train)
    preds_test.append(list(pickle.load(open(path_to_results + model + nameypreds_test, 'rb'))))
        
preds_train=np.array(preds_train).T
preds_test=np.array(preds_test).T    
print(preds_train.shape, preds_test.shape)

((145231L, 35L), (145232L, 35L))


In [6]:
#load in the derived features
X1= np.load('./pickledata/nxtrain_standard_derived0.npy')
xtrain = np.hstack((preds_train, X1))

#load in the derived features for test
X_test1= np.load('./pickledata/nxtest_standard_derived0.npy')
xtest = np.hstack((preds_test, X_test1))
print(xtrain.shape)
print(xtest.shape)

(145231L, 45L)
(145232L, 45L)


In [7]:
y.shape

(145231L,)

In [10]:
max_depth_vals = 10
subsample_vals = 0.9 #0.7:0.805704
colsample_bytree_vals = 0.9
gamma_vals =  0
lambda_vals = 0   
alpha_vals =  1

num_boost_vals = 1000 
eta_vals      =  0.01 
min_child_weight_vals = 3 

rand_state = 870928

In [11]:
bestAUC=0
KFoldNumber=5

kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state = rand_state)
ypreds_test_wl=pd.DataFrame()
ypreds_train_wl=pd.DataFrame()
ypreds_cvindex_wl=pd.DataFrame()
AUCValues=np.zeros(KFoldNumber)

kfoldnum=0

train_predicted=np.zeros(y.shape)

params = {'max_depth':max_depth_vals, 
          'eta':eta_vals, 
          'objective':'binary:logistic', 
          'subsample': subsample_vals,
          'colsample_bytree': colsample_bytree_vals,
          'gamma': gamma_vals,
          'lambda': lambda_vals,
          'alpha' : alpha_vals,
          'eval_metric': 'auc',
          'min_child_weight': min_child_weight_vals 
         }

for train,cv in kfold:
    kfoldnum=kfoldnum+1
    cv_index=np.zeros(y.shape)

    dtrain = xgb.DMatrix(xtrain[train,:], label=y[train])
    clf = xgb.train(params=params,dtrain=dtrain, num_boost_round=num_boost_vals)    

    dfulltrain = xgb.DMatrix(xtrain, label=y)
    preds_fulltrain = clf.predict(dfulltrain)    
    dfulltest = xgb.DMatrix(xtest, label=np.zeros(xtest.shape[0]))
    preds_fulltest = clf.predict(dfulltest)
    preds_cv=preds_fulltrain[cv]
    cv_index[cv]=1

    AUC1=metrics.roc_auc_score(y[cv], preds_cv)

    ypreds_test_wl['CV'+ str(kfoldnum)] = preds_fulltest
    ypreds_train_wl['CV'+ str(kfoldnum)] = preds_fulltrain
    ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
    AUCValues[kfoldnum-1]=float(AUC1)

trainPredict=(ypreds_train_wl*ypreds_cvindex_wl).sum(axis=1)
testPredictMean=ypreds_test_wl.mean(axis=1)
AUC1=metrics.roc_auc_score(y, trainPredict)
print(AUC1)
if AUC1 > bestAUC:
    bestAUC=AUC1
    print('New Best AUC: %f'%bestAUC)

str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename='meta_xgb_'+str1+'_AUC_0p'+ str(int(AUC1.mean()*1e6))+ '.p'
pickle.dump((trainPredict,testPredictMean,ypreds_train_wl,ypreds_test_wl,ypreds_cvindex_wl,AUCValues), 
            open("meta_model/" + filename,'wb'), protocol =2) 

print('Model prediction done!')


0.794108004445
New Best AUC: 0.794108
Model prediction done!
